In [2]:
from qiskit import QuantumCircuit
from pyscf import gto
from jqc.vqe.profile import Profile
from jqc.measure.hamiltonian import hamiltonian
from jqc.simulator.new_state_vector import StateVector

mol = gto.M(atom='H 0 0 0; H 0 0 0.74', basis='sto-3g')
profile = Profile(mol)
hamiltonian = hamiltonian(profile)

qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0, 1)

statevector = StateVector.measure(qc, hamiltonian)



(I, I, I, I)
(Z, I, I, I)
(I, I, Z, I)
(I, Z, I, I)
(I, I, I, Z)
(Z, I, Z, I)
(Z, Z, I, I)
(I, Z, Z, I)
(Z, I, I, Z)
(I, I, Z, Z)
(Y, Y, X, X)
(Y, Y, Y, Y)
(X, X, X, X)
(X, X, Y, Y)
(I, Z, I, Z)


In [19]:
import numpy as np
from scipy.sparse import csr_matrix
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from pyscf import gto
from jqc.vqe.profile import Profile
from jqc.measure.hamiltonian import hamiltonian
from jqc.simulator.state_vector import StateVector

def op_x(state_vector, idx):
    result = np.zeros_like(state_vector)
    non_zero_idx = np.nonzero(state_vector)[0]
    for i, in non_zero_idx:
        new_i = i ^ (1 << idx)
        result[new_i] = value
    return result

def op_y(state_vector, idx):
    result = np.zeros_like(state_vector)
    for i, value in enumerate(state_vector):
        new_i = i ^ (1 << idx)
        if (i >> idx) & 1 == 0:
            result[new_i] = -1j * value
        else:
            result[new_i] = 1j * value
    return result

def op_z(state_vector, idx):
    result = np.copy(state_vector)
    for i, _ in enumerate(state_vector):
        if (i >> idx) & 1:
            result[i] *= -1
    return result

def dict_to_array(sv_dict):
    nqubits = len(list(sv_dict.keys())[0])
    sv_array = np.zeros(2**nqubits, dtype=complex)
    for bit, amplitude in sv_dict.items():
        idx = int(bit, 2)
        sv_array[idx] = amplitude
    return sv_array

qc = QuantumCircuit(4)
qc.h(0)
qc.cx(0, 1)

state_vector = Statevector.from_instruction(qc).data
sparse_sv = csr_matrix(state_vector)
state_vector2 = op_z(state_vector, 1)

print(state_vector)
print(state_vector2)

# measure = sv1.conj().T @ sv2

# print(measure)


[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[ 0.70710678+0.j  0.        +0.j -0.        +0.j -0.70710678+0.j
  0.        +0.j  0.        +0.j -0.        +0.j -0.        +0.j
  0.        +0.j  0.        +0.j -0.        +0.j -0.        +0.j
  0.        +0.j  0.        +0.j -0.        +0.j -0.        +0.j]
